In [ ]:
# Setup: importar bibliotecas, criar diretórios e salvar os CSVs fornecidos em data/raw
import os
from pathlib import Path
import pandas as pd
import re
from datetime import datetime

# Criar estrutura de diretórios
base = Path('data/raw')
base.mkdir(parents=True, exist_ok=True)

# Conteúdo dos arquivos (conforme o enunciado do desafio)
clientes_csv = '''id_cliente,nome,email,telefone,data_nascimento,cidade,estado,data_cadastro
1,João Silva,joao@email.com,11999887766,1985-03-15,São Paulo,SP,2023-01-10
2,Maria Santos,,11888776655,1990-07-22,Rio de Janeiro,RJ,2023-01-15
1,João Silva,joao@email.com,11999887766,1985-03-15,São Paulo,SP,2023-01-10
3,Pedro,pedro@invalid,119999,2000-12-01,Belo Horizonte,MG,2023-02-01
4,,ana@email.com,11777665544,1995-05-30,São Paulo,SP,2023-02-10
'''
produtos_csv = '''id_produto,nome_produto,categoria,preco,estoque,data_criacao,ativo
101,Smartphone XYZ,Eletrônicos,899.99,50,2023-01-01,true
102,Notebook ABC,,1299.99,25,2023-01-05,true
103,Mouse Gamer,Informática,-29.99,100,2023-01-10,true
104,Teclado Mecânico,Informática,199.99,0,2023-01-15,false
105,Smartphone XYZ,Eletrônicos,899.99,50,2023-01-01,true
'''
vendas_csv = '''id_venda,id_cliente,id_produto,quantidade,valor_unitario,valor_total,data_venda,status
1001,1,101,2,899.99,1799.98,2023-03-01,Concluída
1002,2,102,1,1299.99,1299.99,2023-03-02,Pendente
1003,999,103,3,29.99,89.97,2023-03-03,Concluída
1004,1,104,-1,199.99,-199.99,2023-03-04,Cancelada
1005,3,101,1,899.99,899.99,2024-12-31,Processando
'''
logistica_csv = '''id_entrega,id_venda,transportadora,data_envio,data_entrega_prevista,data_entrega_real,status_entrega
2001,1001,Correios,2023-03-02,2023-03-05,2023-03-04,Entregue
2002,1002,Transportadora XYZ,2023-03-03,,2023-03-10,Entregue
2003,1003,Correios,2023-03-04,2023-03-07,,Em Trânsito
2004,1004,,,,,Cancelada
'''
# Salvar arquivos
(base / 'clientes.csv').write_text(clientes_csv, encoding='utf-8')
(base / 'produtos.csv').write_text(produtos_csv, encoding='utf-8')
(base / 'vendas.csv').write_text(vendas_csv, encoding='utf-8')
(base / 'logistica.csv').write_text(logistica_csv, encoding='utf-8')

# Carregar os CSVs em DataFrames
df_clientes = pd.read_csv(base / 'clientes.csv', dtype=str)
df_produtos = pd.read_csv(base / 'produtos.csv', dtype=str)
df_vendas = pd.read_csv(base / 'vendas.csv', dtype=str)
df_logistica = pd.read_csv(base / 'logistica.csv', dtype=str)

# Conversões de tipo necessárias para algumas análises
df_produtos['preco'] = pd.to_numeric(df_produtos['preco'], errors='coerce')
df_produtos['estoque'] = pd.to_numeric(df_produtos['estoque'], errors='coerce').fillna(0).astype(int)
df_vendas['quantidade'] = pd.to_numeric(df_vendas['quantidade'], errors='coerce')
df_vendas['valor_unitario'] = pd.to_numeric(df_vendas['valor_unitario'], errors='coerce')
df_vendas['valor_total'] = pd.to_numeric(df_vendas['valor_total'], errors='coerce')

# Normalizar colunas que serão usadas para junções (ex.: ids como inteiros quando possível)
df_clientes['id_cliente'] = pd.to_numeric(df_clientes['id_cliente'], errors='coerce').astype('Int64')
df_produtos['id_produto'] = pd.to_numeric(df_produtos['id_produto'], errors='coerce').astype('Int64')
df_vendas['id_cliente'] = pd.to_numeric(df_vendas['id_cliente'], errors='coerce').astype('Int64')
df_vendas['id_produto'] = pd.to_numeric(df_vendas['id_produto'], errors='coerce').astype('Int64')
df_vendas['id_venda'] = pd.to_numeric(df_vendas['id_venda'], errors='coerce').astype('Int64')
df_logistica['id_venda'] = pd.to_numeric(df_logistica['id_venda'], errors='coerce').astype('Int64')

print('Arquivos salvos em data/raw e DataFrames carregados:')
print('df_clientes:', df_clientes.shape, 'df_produtos:', df_produtos.shape, 'df_vendas:', df_vendas.shape, 'df_logistica:', df_logistica.shape)

## Análise do Dataset de Clientes

In [ ]:
# Investigação básica
print('--- df_clientes.info() ---')
display(df_clientes.info())
print('
--- df_clientes.describe(include=all) ---')
display(df_clientes.describe(include='all'))

n_total = len(df_clientes)
issues = []  # lista onde vamos acumular problemas para o resumo

def add_issue(dataset, coluna, dimensao, problema, qtd):
    pct = (qtd / n_total * 100) if n_total else 0
    issues.append({
        'Coluna': coluna,
        'Problema Detectado': problema,
2
    })

# 1) Completude: porcentagem de nulos por coluna
pct_nulos = df_clientes.isna().mean() * 100
print('
Percentual de nulos por coluna (clientes):')
print(pct_nulos.round(2))
for col, pct in pct_nulos.items():
    if pct > 0:
        add_issue('clientes', col, 'Completude', f'{pct:.2f}% nulos', int((pct/100)*n_total))

# 2) Unicidade: duplicatas na chave primária e no email
dups_id = df_clientes.duplicated(subset=['id_cliente'], keep=False).sum()
dups_email = df_clientes.duplicated(subset=['email'], keep=False).sum()
print(f'Linhas duplicadas por id_cliente: {dups_id}')
print(f'Linhas duplicadas por email: {dups_email}')
if dups_id > 0:
    add_issue('clientes', 'id_cliente', 'Unicidade', 'ids duplicados', dups_id)
if dups_email > 0:
    add_issue('clientes', 'email', 'Unicidade', 'emails duplicados', dups_email)

if dups_id > 0:
    print('
Exemplo de registros com id_cliente duplicado:')
    display(df_clientes[df_clientes.duplicated(subset=['id_cliente'], keep=False)].sort_values('id_cliente'))

# 3) Validade: emails e telefones
email_regex = re.compile(r'^[-]+@[-]++$')
invalid_emails = df_clientes['email'].fillna('').apply(lambda x: bool(x) and not bool(email_regex.match(x)))
n_invalid_emails = invalid_emails.sum()
print(f'Emails inválidos (clientes): {n_invalid_emails}')
if n_invalid_emails > 0:
    add_issue('clientes', 'email', 'Validade', 'emails inválidos', int(n_invalid_emails))

# Telefones que não têm 11 dígitos numéricos
def is_valid_phone(x):
    if pd.isna(x):
        return False
    s = re.sub(r'', '', str(x))
    return len(s) == 11

invalid_phones = df_clientes['telefone'].apply(lambda x: not is_valid_phone(x))
n_invalid_phones = invalid_phones.sum()
print(f'Telefones inválidos (não 11 dígitos): {n_invalid_phones}')
if n_invalid_phones > 0:
    add_issue('clientes', 'telefone', 'Validade', 'telefone com tamanho inválido', int(n_invalid_phones))

# 4) Consistência: estado com mais de 2 caracteres
invalid_estados = df_clientes['estado'].dropna().apply(lambda x: len(str(x).strip()) != 2)
n_invalid_estados = invalid_estados.sum()
print(f'Estados com tamanho != 2: {n_invalid_estados}')
if n_invalid_estados > 0:
    add_issue('clientes', 'estado', 'Consistência', 'estado com tamanho diferente de 2', int(n_invalid_estados))

# Mostrar issues parciais encontradas até aqui
print('
Issues detectadas (parciais):')
pd.DataFrame(issues).sort_values('Registros Afetados (%)', ascending=False).head(10)

## Análise do Dataset de Produtos

In [ ]:
# Investigação produtos: completude em categoria, preco negativo, estoque zerado
n_total = len(df_produtos)
# Completude: categoria nula
n_categoria_nula = df_produtos['categoria'].isna().sum()
print('Categorias nulas (produtos):', n_categoria_nula)
if n_categoria_nula > 0:
    issues.append({'Dataset':'produtos','Coluna':'categoria','Dimensão da Qualidade':'Completude','Problema Detectado':'categoria nula','Registros Afetados (%)':round(n_categoria_nula/n_total*100,2)})

# Preço negativo
n_preco_neg = (df_produtos['preco'] < 0).sum()
print('Preços negativos (produtos):', int(n_preco_neg))
if n_preco_neg > 0:
    issues.append({'Dataset':'produtos','Coluna':'preco','Dimensão da Qualidade':'Validade','Problema Detectado':'preco < 0','Registros Afetados (%)':round(n_preco_neg/n_total*100,2)})

# Estoque zerado (pode ser aceitável — marcar como informação)
n_estoque_zero = (df_produtos['estoque'] == 0).sum()
print('Produtos com estoque zero:', int(n_estoque_zero))
if n_estoque_zero > 0:
    issues.append({'Dataset':'produtos','Coluna':'estoque','Dimensão da Qualidade':'Acurácia','Problema Detectado':'estoque = 0','Registros Afetados (%)':round(n_estoque_zero/n_total*100,2)})

# Exibir exemplos
display(df_produtos[df_produtos['categoria'].isna()])
display(df_produtos[df_produtos['preco'] < 0])

## Análise do Dataset de Vendas

In [ ]:
# Investigação vendas: integridade referencial, quantidade negativa, valor_total inconsistente, data futura
n_total = len(df_vendas)
# Integridade referencial: id_cliente e id_produto que não existem nas tabelas de referência
clientes_ids = set(df_clientes['id_cliente'].dropna().astype('Int64').astype(int).tolist())
produtos_ids = set(df_produtos['id_produto'].dropna().astype('Int64').astype(int).tolist())

# ids de vendas que referenciam clientes/produtos inexistentes
vendas_cliente_nao_existe = df_vendas[~df_vendas['id_cliente'].isin(clientes_ids)]
vendas_produto_nao_existe = df_vendas[~df_vendas['id_produto'].isin(produtos_ids)]
print('Vendas com id_cliente inexistente:', len(vendas_cliente_nao_existe))
print('Vendas com id_produto inexistente:', len(vendas_produto_nao_existe))
if len(vendas_cliente_nao_existe) > 0:
    issues.append({'Dataset':'vendas','Coluna':'id_cliente','Dimensão da Qualidade':'Consistência','Problema Detectado':'id_cliente não existe em clientes','Registros Afetados (%)':round(len(vendas_cliente_nao_existe)/n_total*100,2)})
if len(vendas_produto_nao_existe) > 0:
    issues.append({'Dataset':'vendas','Coluna':'id_produto','Dimensão da Qualidade':'Consistência','Problema Detectado':'id_produto não existe em produtos','Registros Afetados (%)':round(len(vendas_produto_nao_existe)/n_total*100,2)})

# Quantidade negativa
n_qtd_neg = (df_vendas['quantidade'] < 0).sum()
print('Vendas com quantidade negativa:', int(n_qtd_neg))
if n_qtd_neg > 0:
    issues.append({'Dataset':'vendas','Coluna':'quantidade','Dimensão da Qualidade':'Validade','Problema Detectado':'quantidade <= 0','Registros Afetados (%)':round(n_qtd_neg/n_total*100,2)})

# Valor total inconsistente: comparar quantidade * valor_unitario com valor_total (tolerância pequena para arredondamento)
tol = 0.01
calc_total = (df_vendas['quantidade'] * df_vendas['valor_unitario']).round(2)
inconsistent_total = (~(calc_total - df_vendas['valor_total']).abs().le(tol))
n_inconsistent = inconsistent_total.sum()
print('Vendas com valor_total inconsistente:', int(n_inconsistent))
if n_inconsistent > 0:
    issues.append({'Dataset':'vendas','Coluna':'valor_total','Dimensão da Qualidade':'Consistência','Problema Detectado':'valor_total != quantidade * valor_unitario','Registros Afetados (%)':round(n_inconsistent/n_total*100,2)})

# Data de venda no futuro
df_vendas['data_venda_parsed'] = pd.to_datetime(df_vendas['data_venda'], errors='coerce')
hoje = pd.Timestamp.now().normalize()
n_data_futura = (df_vendas['data_venda_parsed'] > hoje).sum()
print('Vendas com data_venda no futuro:', int(n_data_futura))
if n_data_futura > 0:
    issues.append({'Dataset':'vendas','Coluna':'data_venda','Dimensão da Qualidade':'Temporalidade','Problema Detectado':'data_venda no futuro','Registros Afetados (%)':round(n_data_futura/n_total*100,2)})

# Exibir exemplos de problemas detectados
display(vendas_cliente_nao_existe)
display(df_vendas[inconsistent_total])

## Análise do Dataset de Logística

In [ ]:
# Investigação logística: data_envio nula, data_entrega_prevista nula e consistência de datas
n_total = len(df_logistica)
n_data_envio_nula = df_logistica['data_envio'].isna().sum()
n_data_entrega_prev_nula = df_logistica['data_entrega_prevista'].isna().sum()
print('Logística - data_envio nula:', int(n_data_envio_nula))
print('Logística - data_entrega_prevista nula:', int(n_data_entrega_prev_nula))
if n_data_envio_nula > 0:
    issues.append({'Dataset':'logistica','Coluna':'data_envio','Dimensão da Qualidade':'Completude','Problema Detectado':'data_envio nula','Registros Afetados (%)':round(n_data_envio_nula/n_total*100,2)})
if n_data_entrega_prev_nula > 0:
    issues.append({'Dataset':'logistica','Coluna':'data_entrega_prevista','Dimensão da Qualidade':'Completude','Problema Detectado':'data_entrega_prevista nula','Registros Afetados (%)':round(n_data_entrega_prev_nula/n_total*100,2)})

# Exibir amostras
display(df_logistica[df_logistica['data_envio'].isna()])
display(df_logistica[df_logistica['data_entrega_prevista'].isna()])

## Sumário e Priorização

In [ ]:
# Construir DataFrame de resumo programaticamente a partir da lista `issues`
df_summary = pd.DataFrame(issues)
# Normalizar coluna de porcentagem caso exista como texto e garantir tipo numérico
if 'Registros Afetados (%)' in df_summary.columns:
    df_summary['Registros Afetados (%)'] = pd.to_numeric(df_summary['Registros Afetados (%)'], errors='coerce').fillna(0)

# Ordenar por impacto decrescente (porcentagem de registros afetados)
df_summary = df_summary.sort_values('Registros Afetados (%)', ascending=False).reset_index(drop=True)

print('Resumo de problemas detectados (ordenado por criticidade):')
display(df_summary)

# Comentários técnicos sobre escolhas de análise:
# - Usamos df.duplicated(subset=['id_cliente']) para detectar duplicatas na chave primária, pois
#   (keep=False retorna todos os registros duplicados). Soma de True indica total de linhas envolvidas em duplicidade.
#   presentes nas tabelas de referência para detectar referências inexistentes de forma eficiente.
#   derivados de problemas de representação decimal ou arredondamento.
100
#   de tamanhos diferentes diretas e interpretáveis.
,
,
,
,
,

: {
: {
: 
,
: 
3

: {
: 
,
: 
3

: 4,
: 5